In [4]:
import re, os
import unicodedata
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import nltk.sentiment

from requests import get
from bs4 import BeautifulSoup

from wordcloud import WordCloud


plt.rc('figure', figsize=(13, 7))
plt.style.use('seaborn-darkgrid')

In [5]:
import acquire

In [6]:
def get_all_repository_urls(url): # repository url
    # Get max page.
    response = get(url)
    soup = BeautifulSoup(response.text)
    print('Finding max page for repositories...')
    max_page = int(soup.find('div', role='navigation').text[-6])
    print(f'Max page found: {max_page}')
    page = 1
    repository_links = []
    print('Starting loop...')
    for n in range(max_page):
        print(n+1, 'iteration')
        print(f'Pulling data from {url}')
        # Reset soup.
        response = get(url)
        soup = BeautifulSoup(response.text)
        # Get all the repositories from the page.
        repositories = soup.find_all('a', itemprop='name codeRepository') 
        print('Fetching links for repositories...')
        for repo in repositories:
            repository_links.append(repo.get('href'))
        git = 'https://github.com/'
        next_page = soup.find('a', class_='next_page').get('href')[:-1]
        ## Use this line of code to get the url for the next page.
        if page <= 4:
            url = git + next_page + str(page + 1)
            page += 1
        else:
            return repository_links


In [7]:
repository_links = get_all_repository_urls('https://github.com/orgs/apple/repositories')

Finding max page for repositories...
Max page found: 6
Starting loop...
1 iteration
Pulling data from https://github.com/orgs/apple/repositories
Fetching links for repositories...
2 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=2
Fetching links for repositories...
3 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=3
Fetching links for repositories...
4 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=4
Fetching links for repositories...
5 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=5
Fetching links for repositories...


In [8]:
repository_links

['/apple/llvm-project',
 '/apple/swift',
 '/apple/sourcekit-lsp',
 '/apple/foundationdb',
 '/apple/swift-docc',
 '/apple/swift-protobuf',
 '/apple/swift-llbuild',
 '/apple/swift-syntax',
 '/apple/swift-package-manager',
 '/apple/swift-source-compat-suite',
 '/apple/swift-markdown',
 '/apple/swift-docc-symbolkit',
 '/apple/swift-docc-render-artifact',
 '/apple/swift-lmdb',
 '/apple/swift-tools-support-core',
 '/apple/swift-driver',
 '/apple/indexstore-db',
 '/apple/swift-stress-tester',
 '/apple/swift-xcode-playground-support',
 '/apple/swift-integration-tests',
 '/apple/swift-cmark',
 '/apple/swift-corelibs-libdispatch',
 '/apple/swift-corelibs-foundation',
 '/apple/swift-corelibs-xctest',
 '/apple/swift-experimental-string-processing',
 '/apple/swift-docker',
 '/apple/swift-installer-scripts',
 '/apple/swift-llbuild2',
 '/apple/swift-docc-render',
 '/apple/swift-evolution',
 '/apple/servicetalk',
 '/apple/swift-nio-ssh',
 '/apple/password-manager-resources',
 '/apple/swift-nio',
 '/ap

In [9]:
import acquire_jg

In [10]:
corpus = acquire_jg.scrape_github_data()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)